In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('./data/train.csv', parse_dates=['date'])
test = pd.read_csv('./data/test.csv', parse_dates=['date'])
stores = pd.read_csv('./data/stores.csv')
transactions = pd.read_csv('./data/transactions.csv', parse_dates=['date'])
oil = pd.read_csv('./data/oil.csv', parse_dates=['date'])
holidays = pd.read_csv('./data/holidays_events.csv', parse_dates=['date'])

In [3]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
train.family.unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [5]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [6]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
stores.type.unique()

array(['D', 'B', 'C', 'E', 'A'], dtype=object)

In [8]:
stores.cluster.unique()

array([13,  8,  9,  4,  6, 15,  7,  3, 12, 16,  1, 10,  2,  5, 11, 14, 17])

In [9]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [10]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [11]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [12]:
holidays.type.unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [13]:
holidays[holidays.type == 'Work Day']

,date,type,locale,locale_name,description,transferred
42,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
43,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
149,2014-12-20,Work Day,National,Ecuador,Recupero Puente Navidad,False
161,2015-01-10,Work Day,National,Ecuador,Recupero Puente Primer dia del ano,False
283,2016-11-12,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [14]:
holidays.locale.unique()

array(['Local', 'Regional', 'National'], dtype=object)

In [15]:
holidays[holidays.locale == 'Local'].locale_name.unique()

array(['Manta', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo', 'Guaranda',
       'Latacunga', 'Machala', 'Santo Domingo', 'El Carmen', 'Cayambe',
       'Esmeraldas', 'Ambato', 'Ibarra', 'Quevedo', 'Quito', 'Loja',
       'Salinas', 'Guayaquil'], dtype=object)

In [16]:
train.shape

(3000888, 6)

In [17]:
result0 = train.merge(
    stores,
    on='store_nbr',
)
result0.shape

(3000888, 10)

In [18]:
transactions.shape

(83488, 3)

In [19]:
result1 = result0.merge(
    transactions,
    on=['date', 'store_nbr'],
    how='left',
)
result1.shape

(3000888, 11)

In [20]:
result2 = result1.merge(
    oil,
    on='date',
    how='left',
)
result2.shape

(3000888, 12)

In [21]:
events = holidays[(holidays.transferred == False) & (holidays.type != 'Work Day')].copy()
events['holiday'] = 1
events.shape

(333, 7)

In [22]:
national_events = events[events.locale == 'National']
national_events.shape

(161, 7)

In [23]:
national_events.date.value_counts()

2012-12-31    2
2016-05-01    2
2016-05-08    2
2016-05-07    2
2014-12-26    2
             ..
2014-10-10    1
2014-11-02    1
2014-11-03    1
2014-11-28    1
2017-12-26    1
Name: date, Length: 155, dtype: int64

In [24]:
national_event_series = national_events.groupby('date').holiday.any()
national_event_df = pd.DataFrame({'date': national_event_series.index, 'is_holiday': national_event_series.values})
national_event_df.head()

,date,is_holiday
0,2012-08-10,True
1,2012-10-12,True
2,2012-11-02,True
3,2012-11-03,True
4,2012-12-21,True


In [25]:
regional_events = events[events.locale == 'Regional']
regional_events.shape

(24, 7)

In [26]:
regional_event_series = regional_events.groupby(['date', 'locale_name']).holiday.any()
data = []
for (dt, locale_name), is_holiday in regional_event_series.items():
    data.append({'date': dt, 'state': locale_name, 'is_holiday': is_holiday})
regional_event_df = pd.DataFrame(data)
regional_event_df.head()

,date,state,is_holiday
0,2012-04-01,Cotopaxi,True
1,2012-06-25,Imbabura,True
2,2012-11-06,Santo Domingo de los Tsachilas,True
3,2012-11-07,Santa Elena,True
4,2013-04-01,Cotopaxi,True


In [27]:
local_events = events[events.locale == 'Local']
local_events.shape

(148, 7)

In [28]:
local_event_series = local_events.groupby(['date', 'locale_name']).holiday.any()
data = []
for (dt, locale_name), is_holiday in local_event_series.items():
    data.append({'date': dt, 'city': locale_name, 'is_holiday': is_holiday})
local_event_df = pd.DataFrame(data)
local_event_df.head()

,date,city,is_holiday
0,2012-03-02,Manta,True
1,2012-04-12,Cuenca,True
2,2012-04-14,Libertad,True
3,2012-04-21,Riobamba,True
4,2012-05-12,Puyo,True


In [29]:
result3 = result2.merge(
    national_event_df,
    on='date',
    how='left',
).merge(
    regional_event_df,
    on=['date', 'state'],
    how='left',
).merge(
    local_event_df,
    on=['date', 'city'],
    how='left',
)
result3.shape

(3000888, 15)

In [30]:
result3.is_holiday.fillna(False, inplace=True)
result3.is_holiday_x.fillna(False, inplace=True)
result3.is_holiday_y.fillna(False, inplace=True)

result3['is_holiday'] = np.max(result3[['is_holiday', 'is_holiday_x', 'is_holiday_y']].values)
result3.drop(['is_holiday_x', 'is_holiday_y'], axis=1, inplace=True)

In [31]:
result3.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions,dcoilwtico,is_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,True
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,True
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,True
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,True
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,True


In [32]:
result3['year'] = result3.date.apply(lambda x: x.year)
result3['month'] = result3.date.apply(lambda x: x.month)
result3['day'] = result3.date.apply(lambda x: x.day)
result3['day_of_week'] = result3.date.apply(lambda x: x.day_of_week)

In [33]:
result3[~result3.dcoilwtico.isna()].shape[0] / result3.shape[0]

0.6906175771971497

In [34]:
result3['oil_na'] = result3.dcoilwtico.map(np.isnan)
result3['oil_mean'] = result3.groupby(['year', 'month']).dcoilwtico.transform('mean')
result3.dcoilwtico.fillna(0, inplace=True)
result3['oil_price_tmp'] = result3.oil_na * result3.oil_mean
result3['dcoilwtico'] = result3.dcoilwtico + result3.oil_price_tmp * result3.oil_na
result3.drop(['oil_na', 'oil_mean', 'oil_price_tmp'], axis=1, inplace=True)

In [35]:
result3[~result3.dcoilwtico.isna()].shape[0] / result3.shape[0]

1.0

In [36]:
result3[~result3.transactions.isna()].shape[0] / result3.shape[0]

0.9180962435119204

In [37]:
result3[np.isnan(result3.transactions.values)].sales.value_counts()

0.000       242536
1.000          160
2.000          146
3.000          103
4.000           83
             ...  
3247.000         1
596.125          1
295.181          1
1033.000         1
1588.439         1
Name: sales, Length: 1541, dtype: int64

In [38]:
result3.transactions.fillna(-1, inplace=True)

In [39]:
result3.to_csv('data/train_merged.csv', index=False)

In [41]:
test.shape

(28512, 5)

In [40]:
test_result = test.merge(
    stores,
    on='store_nbr',
).merge(
    transactions,
    on=['date', 'store_nbr'],
    how='left',
).merge(
    oil,
    on='date',
    how='left',
).merge(
    national_event_df,
    on='date',
    how='left',
).merge(
    regional_event_df,
    on=['date', 'state'],
    how='left',
).merge(
    local_event_df,
    on=['date', 'city'],
    how='left',
)

In [42]:
test_result.shape

(28512, 14)

In [43]:
test_result.is_holiday.fillna(False, inplace=True)
test_result.is_holiday_x.fillna(False, inplace=True)
test_result.is_holiday_y.fillna(False, inplace=True)

test_result['is_holiday'] = np.max(test_result[['is_holiday', 'is_holiday_x', 'is_holiday_y']].values)
test_result.drop(['is_holiday_x', 'is_holiday_y'], axis=1, inplace=True)

In [45]:
test_result['year'] = test_result.date.apply(lambda x: x.year)
test_result['month'] = test_result.date.apply(lambda x: x.month)
test_result['day'] = test_result.date.apply(lambda x: x.day)
test_result['day_of_week'] = test_result.date.apply(lambda x: x.day_of_week)

In [46]:
test_result['oil_na'] = test_result.dcoilwtico.map(np.isnan)
test_result['oil_mean'] = test_result.groupby(['year', 'month']).dcoilwtico.transform('mean')
test_result.dcoilwtico.fillna(0, inplace=True)
test_result['oil_price_tmp'] = test_result.oil_na * test_result.oil_mean
test_result['dcoilwtico'] = test_result.dcoilwtico + test_result.oil_price_tmp * test_result.oil_na
test_result.drop(['oil_na', 'oil_mean', 'oil_price_tmp'], axis=1, inplace=True)

In [47]:
test_result.transactions.fillna(-1, inplace=True)

In [48]:
test_result.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,transactions,dcoilwtico,is_holiday,year,month,day,day_of_week
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,-1.0,46.8,True,2017,8,16,2
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,-1.0,46.8,True,2017,8,16,2
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,-1.0,46.8,True,2017,8,16,2
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,-1.0,46.8,True,2017,8,16,2
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,-1.0,46.8,True,2017,8,16,2


In [49]:
test_result.to_csv('data/test_merged.csv')